In [2]:
# in order to make the source code easy to read and modify I've placed those modules in a separate files    
import DataExtraction   # this module is responsible for extractin the email's body from the mailbox [without signature]
import DataAnalysis     # this module is responsible for parsing the email's body and extracting the required data

def main():
    # Specify which mailbox/store to read, exactly matching 'store.DisplayName'
        # this part of the source code can be modified to read from a different mailbox or subfolder
        # in case we have to run it on different user's machine or mailbox.
    mailbox_name    = "n.yumer@my.bpp.com"  
    subfolder_name  = "ROMIC"

    # Read emails from the subfolder
    all_bodies = DataExtraction.read_outlook_subfolder_stores(mailbox_name, subfolder_name)

    # #Debug section to see what dataextraction is returning
    # print("All bodies (no signature):")
    # for i, body_text in enumerate(all_bodies, start=1):
    #     print(f"Body #{i}:\n{body_text}")
    #     print("-" * 40)
    # #Debug section to see what dataextraction is returning

    parsed_results = DataAnalysis.parse_emails(all_bodies)
    print("RESULTS:")
    print(f"  Monitors: {parsed_results['monitors']}")

    # [NEW] Print the recognized desktop models
    print("  Desktops:")
    for model, qty in parsed_results['desktops'].items():
        print(f"    Model {model}: {qty}")

    # [NEW] Print the recognized laptop models
    print("  Laptops:")
    for model, qty in parsed_results['laptops'].items():
        print(f"    Model {model}: {qty}")

    # [NEW] Print the recognized phone models
    print("  Phones:")
    for model, qty in parsed_results['phones'].items():
        print(f"    Model {model}: {qty}")

    print("  Bags:")
    for bag_type, qty in parsed_results['bags'].items():
        print(f"    {bag_type} bag: {qty}")

    print("  Chargers:")
    for charger_label, qty in parsed_results['chargers'].items():
        print(f"    {charger_label}: {qty}")

    print(f"  Docks: {parsed_results['docks']}")

    print("  Headsets:")
    for headset_label, qty in parsed_results['headsets'].items():
        print(f"    {headset_label}: {qty}")

    # [NEW] Print the unparsable lines with sender and time
    print("  Unparsable Lines:")
    for unparsable in parsed_results['unparsable_lines']:
        line = unparsable['line']
        sender = unparsable['sender']
        time = unparsable['received_time']
        print(f"    Line: \"{line}\", Sender: {sender}, Received Time: {time}")
if __name__ == "__main__":
    main()



RESULTS:
  Monitors: 10
  Desktops:
    Model 3000: 1
  Laptops:
    Model 5540: 1
  Phones:
  Bags:
  Chargers:
  Docks: 0
  Headsets:
  Unparsable Lines:
    Line: "Phone collected S23 A35", Sender: Nihat R. Yumer, Received Time: 2024-12-29 16:39:46.527000+00:00
    Line: "Lapotop 5666", Sender: Nihat R. Yumer, Received Time: 2024-12-29 16:39:46.527000+00:00
    Line: "PC x2 3111", Sender: Nihat R. Yumer, Received Time: 2024-12-29 16:39:46.527000+00:00
    Line: "Asset details of devices collected (if required)     Asset 076975", Sender: Melay Connor, Received Time: 2024-12-20 15:37:20.598000+00:00
    Line: "Hi,", Sender: Baramasimbe Ronnie, Received Time: 2024-12-20 12:12:07.829000+00:00
    Line: "I have booked out the following items:", Sender: Baramasimbe Ronnie, Received Time: 2024-12-20 12:12:07.829000+00:00
    Line: "With", Sender: Baramasimbe Ronnie, Received Time: 2024-12-20 12:12:07.829000+00:00
